In [1]:
from src.data.transforms import Rotate
import PIL.Image as Image
import jax
import jax.numpy as jnp
import numpy as np

#test = jnp.array(np.array(Image.open('tests/data/test.png').convert('RGB')).transpose(2, 1, 0).reshape(1, 3, 224, 224))
test = jnp.array(np.random.randint(0, 255, (32, 3, 224, 224), dtype=np.uint8))
key = jax.random.PRNGKey(0)
s = Rotate((0, 360))

In [2]:
s(test)

Array([[[[  0,   0,   0, ...,   0,   0,   0],
         [  0,  33,  88, ..., 124, 231,  57],
         [  0,  33, 130, ...,  27, 157, 166],
         ...,
         [  0, 216, 211, ..., 209, 147, 241],
         [  0,  34,   0, ..., 190,  24,  86],
         [  0,  78, 174, ..., 189, 170,  81]],

        [[  0,   0,   0, ...,   0,   0,   0],
         [  0,  40, 155, ..., 209, 179, 141],
         [  0, 171,  79, ..., 141, 147, 219],
         ...,
         [  0, 133, 149, ...,  27,  46, 231],
         [  0, 187, 173, ..., 203, 199,  69],
         [  0,  59, 153, ..., 198,  19, 138]],

        [[  0,   0,   0, ...,   0,   0,   0],
         [  0, 172,  64, ..., 203,  67, 218],
         [  0, 205,  57, ..., 168, 224,  46],
         ...,
         [  0,  47, 242, ..., 195, 140, 251],
         [  0,  15, 196, ...,  33, 126,  93],
         [  0, 250, 188, ..., 222, 123,  12]]],


       [[[  0,   0,   0, ...,   0,   0,   0],
         [  0, 142, 240, ..., 197, 117, 188],
         [  0,  23, 220, ..., 

In [3]:
s(test)

Array([[[[  0,   0,   0, ...,   0,   0,   0],
         [  0,  33,  88, ..., 124, 231,  57],
         [  0,  33, 130, ...,  27, 157, 166],
         ...,
         [  0, 216, 211, ..., 209, 147, 241],
         [  0,  34,   0, ..., 190,  24,  86],
         [  0,  78, 174, ..., 189, 170,  81]],

        [[  0,   0,   0, ...,   0,   0,   0],
         [  0,  40, 155, ..., 209, 179, 141],
         [  0, 171,  79, ..., 141, 147, 219],
         ...,
         [  0, 133, 149, ...,  27,  46, 231],
         [  0, 187, 173, ..., 203, 199,  69],
         [  0,  59, 153, ..., 198,  19, 138]],

        [[  0,   0,   0, ...,   0,   0,   0],
         [  0, 172,  64, ..., 203,  67, 218],
         [  0, 205,  57, ..., 168, 224,  46],
         ...,
         [  0,  47, 242, ..., 195, 140, 251],
         [  0,  15, 196, ...,  33, 126,  93],
         [  0, 250, 188, ..., 222, 123,  12]]],


       [[[  0,   0,   0, ...,   0,   0,   0],
         [  0, 142, 240, ..., 197, 117, 188],
         [  0,  23, 220, ..., 